In [51]:
import pandas as pd
import numpy as np
import difflib
import os

In [66]:
cd C:\Users\Diego\Documents\DTU\Spring2024\MBML\european_vote_analysis

C:\Users\Diego\Documents\DTU\Spring2024\MBML\european_vote_analysis


In [236]:
data_european = pd.read_csv('data/raw/COMEPELDA_aggregate_v1.00.csv')
data_national = pd.read_csv('data/raw/1999-2019_CHES_dataset_means.csv')
country_codes = pd.read_csv('data/raw/countries.csv')  
party_names = pd.read_csv('data/raw/partyNames_ncv2.csv')  

In [237]:
# Add country codes to national data
data_european = pd.merge(data_european, country_codes, left_on='cName', right_on='Country', how='left')
# drop 2006, 2007, 2013 years due to particular countries elections
data_european = data_european.drop(data_european[data_european['ElYear'].isin([2006, 2007, 2013])].index)

In [238]:
# add country codes to european data
data_national = pd.merge(data_national, country_codes, left_on='country', right_on='ID', how='left')

In [239]:
data_national = pd.merge(data_national, party_names, left_on='party_id', right_on='partyID', how='left')

In [ ]:
test_data = pd.merge(data_european, data_national, left_on=["Country", 'partyName'], right_on=["cName", 'pName'], how='outer', indicator=True)

In [97]:
T1 = test_data[test_data['_merge'] == "left_only"]
T2 = test_data[test_data['_merge'] == "right_only"]
T3 = test_data[test_data['_merge'] == "both"]

In [158]:
T1.describe()

,country,eastwest,eumember,year,expert,party_id,cmp_id,vote,seat,electionyear,...,pdFlagNofCand,cSeats,pcSeats,pdSeats,IndepCand01,cAddLisbon,pAddLisbon,pAlliance01,IDauxElec,ID_y
count,1156.000000,1156.000000,1156.000000,1156.000000,1156.000000,1156.000000,928.000000,1098.000000,1093.000000,1156.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
mean,14.448962,0.648789,0.928201,2009.638408,11.804498,1453.041522,49951.885776,11.599754,12.202379,2008.301903,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
std,9.757492,0.477555,0.258267,6.754755,4.684488,974.625766,28885.510551,11.365716,14.055509,6.803083,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
min,1.000000,0.000000,0.000000,1999.000000,2.000000,102.000000,11110.000000,0.010000,0.000000,1996.000000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
25%,6.000000,0.000000,1.000000,2002.000000,8.000000,612.750000,23200.750000,3.200000,1.800000,2002.000000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
50%,11.000000,1.000000,1.000000,2010.000000,11.000000,1108.000000,41470.500000,7.130000,7.000000,2008.000000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
75%,24.000000,1.000000,1.000000,2014.000000,15.000000,2402.000000,82413.000000,16.800000,17.299999,2014.000000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
max,40.000000,1.000000,1.000000,2019.000000,27.000000,4009.000000,97952.000000,55.040000,64.599998,2019.000000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [99]:
T2.describe()

,country,eastwest,eumember,year,expert,party_id,cmp_id,vote,seat,electionyear,...,pdFlagNofCand,cSeats,pcSeats,pdSeats,IndepCand01,cAddLisbon,pAddLisbon,pAlliance01,IDauxElec,ID_y
count,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1512.000000,1512.000000,1512.000000,1512.000000,1512.000000,1512.000000,1512.000000,1512.000000,1512.000000,1512.000000
mean,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.001984,53.291005,6.937831,1.605820,0.011905,0.291005,0.037698,0.111111,429.771164,13.554233
std,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.044514,28.279169,9.647944,2.633474,0.108493,0.599999,0.193971,0.314374,258.284368,9.265678
min,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.000000,3.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,4.000000,1.000000
25%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.000000,22.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,199.000000,6.000000
50%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.000000,54.000000,3.000000,1.000000,0.000000,0.000000,0.000000,0.000000,346.000000,11.000000
75%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.000000,75.000000,9.000000,2.000000,0.000000,0.000000,0.000000,0.000000,676.000000,24.250000
max,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1.000000,99.000000,43.000000,27.000000,1.000000,4.000000,2.000000,1.000000,849.000000,40.000000


In [100]:
T3.describe()

,country,eastwest,eumember,year,expert,party_id,cmp_id,vote,seat,electionyear,...,pdFlagNofCand,cSeats,pcSeats,pdSeats,IndepCand01,cAddLisbon,pAddLisbon,pAlliance01,IDauxElec,ID_y
count,294.000000,294.000000,294.000000,294.000000,294.000000,294.000000,241.000000,276.00000,276.000000,294.000000,...,294.0,294.000000,294.000000,294.000000,294.0,294.0,294.0,294.000000,294.000000,294.000000
mean,14.306122,0.683673,0.986395,2008.380952,10.969388,1437.656463,54632.203320,14.41087,15.766667,2005.928571,...,0.0,52.731293,7.108844,1.819728,0.0,0.0,0.0,0.088435,698.843537,14.306122
std,8.897300,0.465835,0.116044,7.219932,4.520233,888.813767,26738.557581,14.22145,18.584006,7.096483,...,0.0,27.749721,8.346285,3.390131,0.0,0.0,0.0,0.284411,213.505240,8.897300
min,1.000000,0.000000,0.000000,1999.000000,4.000000,102.000000,11520.000000,0.20000,0.000000,1996.000000,...,0.0,6.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000,4.000000,1.000000
25%,8.000000,0.000000,1.000000,1999.000000,7.000000,808.000000,32720.000000,2.87500,0.500000,1997.000000,...,0.0,22.000000,2.000000,0.000000,0.0,0.0,0.0,0.000000,668.000000,8.000000
50%,11.000000,1.000000,1.000000,2014.000000,10.000000,1107.000000,51320.000000,8.40000,6.100000,2011.000000,...,0.0,51.000000,4.000000,1.000000,0.0,0.0,0.0,0.000000,833.000000,11.000000
75%,26.000000,1.000000,1.000000,2014.000000,15.000000,2601.000000,92210.000000,28.65000,32.500000,2011.000000,...,0.0,84.000000,7.000000,2.000000,0.0,0.0,0.0,0.000000,837.000000,26.000000
max,40.000000,1.000000,1.000000,2014.000000,22.000000,4005.000000,97951.000000,54.80000,63.599998,2014.000000,...,0.0,99.000000,33.000000,33.000000,0.0,0.0,0.0,1.000000,849.000000,40.000000


In [115]:
data_european["alt_partyName"] = data_european[data_european["Country"] == "Belgium"]["partyName"].apply(lambda x: difflib.get_close_matches(x, data_national[data_national["cName"] == "Belgium"]["pName"]))

In [122]:
data_european["alt_partyName"] = data_european["partyName"].astype(str).apply(lambda x: difflib.get_close_matches(x, data_national["pName"].astype(str)))

In [240]:
year_corr = {1999: 1999, 
 2004: 2002,
 2009: 2006,
 2014: 2014}
eu_unique_columns = ["Abbreviation", "ElYear", "pName"]
data_unique_columns = ["Abbreviation", "year", "partyName"]
parties_data_eu = data_european[eu_unique_columns].drop_duplicates()
parties_data_nat = data_national[data_unique_columns].drop_duplicates()

In [241]:
for row in parties_data_eu.itertuples():
    y = parties_data_nat[(parties_data_nat["Abbreviation"] == row.Abbreviation) & 
                         (parties_data_nat["year"] == year_corr[row.ElYear])]["partyName"].astype(str)
    x = row.pName
    result = difflib.get_close_matches(x, y, n=1)
    if result:
        parties_data_eu.loc[row.Index, "correspondance"] = result[0]
    else: 
        parties_data_eu.loc[row.Index, "correspondance"] = np.nan

In [244]:
parties_data_eu.correspondance.isna().sum()

214

In [245]:
parties_data_eu.to_csv("data/processed/party_correspondance.csv", index=False)

In [99]:
T3[["Country_x", "partyName", "cName", "pName"]].drop_duplicates()

,Country_x,partyName,cName,pName
42,Belgium,Parti Socialiste,Belgium,Parti Socialiste
102,Denmark,Fremskridtspartiet,Denmark,Fremskridtspartiet
103,Denmark,Kristeligt Folkeparti,Denmark,Kristeligt Folkeparti
105,Denmark,Socialistisk Folkeparti,Denmark,Socialistisk Folkeparti
129,Denmark,Socialdemokraterne,Denmark,Socialdemokraterne
...,...,...,...,...
3900,Malta,Partit Nazzjonalista,Malta,Partit Nazzjonalista
3923,Cyprus,Dimokratikó Kómma,Cyprus,Dimokratikó Kómma
3929,Cyprus,Evropaiko Komma,Cyprus,Evropaiko Komma
3930,Cyprus,Kinima Sosialdimokraton EDEK,Cyprus,Kinima Sosialdimokraton EDEK


In [98]:
T1[["Country_x", "partyName", "cName", "pName"]].drop_duplicates()

,Country_x,partyName,cName,pName
0,Belgium,Front National,NaN,NaN
2,Belgium,Christelijke Volkspartij; Christen-Democratisc...,NaN,NaN
8,Belgium,Open Vlaamse Liberalen en Democraten,NaN,NaN
14,Belgium,Parti Réformateur Libéral; Mouvement Réformateur,NaN,NaN
20,Belgium,Volksunie; Nieuw-Vlaamse Alliantie,NaN,NaN
...,...,...,...,...
3919,Cyprus,Kinima Oikologon Perivallontiston,NaN,NaN
3921,Cyprus,Dimokratikós Sinagermós,NaN,NaN
3940,Cyprus,Symmachía Politón,NaN,NaN
3941,Cyprus,Kinima Allilengyu,NaN,NaN
